# OSSF demo

In this notebook, we will demonstrate how **Legend** models can be interpreted on databricks with minimum development overhead. Although we could directly query the underlying tables through the spark SQL API and its multiple joins and transformations (see below), we can benefit from the model created through the **Legend** framework and access its logical representation (mapping to a legend entity).

In [0]:
%sql
SELECT
  `id`,
  `currency`,
  `reporting_entity`,
  `product`,
  `sub_product`,
  `market_value`,
  `lendable_value`,
  `forward_start_amount`,
  `forward_start_bucket`,
  `collateral_class`,
  `treasury_control`,
  `accounting_designation`,
  `encumbrance_type`,
  `internal_counterparty`,
  `business_line`
FROM inflows.assets

## LEGEND
With our **Legend** model packaged as JAR and included as a cluster dependency, we can easily access each of its underlying entities, create tables programmatically or execute queries according to legend specifications. We show below how to load our model and access a given entity (a logical model mapped to a physical table)

In [0]:
%scala
import org.finos.legend.spark.LegendClasspathLoader
val legend = LegendClasspathLoader.loadResources()

import org.finos.legend.spark.LegendClasspathLoader
legend: org.finos.legend.spark.Legend = org.finos.legend.spark.Legend@32a2c203

Accessing the underlying generated SQL code from pure to databricks SQL...

In [0]:
%scala
legend.generateSql("lcr::lakehouse::assetMapping")

res2: String = select `root`.sub_product as `subProduct`, `root`.reporting_entity as `reportingEntity`, `root`.lendable_value as `lendableValue`, `root`.accounting_designation as `accountingDesignation`, `root`.treasury_control as `treasuryControl`, `root`.encumbrance_type as `encumbranceType`, `root`.business_line as `businessLine`, `root`.collateral_class as `collateralClass`, `root`.id as `id`, `root`.market_value as `marketValue`, `root`.forward_start_bucket as `forwardStartBucket`, `root`.currency as `currency`, `root`.internal_counterparty as `internalCounterparty`, `root`.converted as `converted`, `root`.forward_start_amount as `forwardStartAmount`, `root`.product as `product` from inflows.assets as `root` where `root`.reporting_entity is not null and `root`.product is not null and `root`.converted is not null and `root`.treasury_control is not null and `root`.lendable_value is not null and (`root`.currency is null or `root`.currency in ('USD', 'EUR', 'GBP', 'CHF', 'JPY', 'AUD', 'CAD')) and `root`.currency is not null and `root`.market_value is not null and `root`.id is not null and `root`.business_line is not null and (`root`.product is null or `root`.product in ('UnencumberedAssets', 'Capacity', 'UnrestrictedReserveBalances', 'RestrictedReserveBalances', 'UnsettledAssetPurchases', 'ForwardAssetPurchases', 'EncumberedAssets')) and (`root`.collateral_class is null or `root`.collateral_class in ('a_0_Q', 'a_1_Q', 'a_2_Q', 'a_3_Q', 'a_4_Q', 'a_5_Q', 's_1_Q', 's_2_Q', 's_3_Q', 's_4_Q', 's_5_Q', 's_6_Q', 's_7_Q', 'cB_1_Q', 'cB_2_Q', 'cB_3_Q', 'g_1_Q', 'g_2_Q', 'g_3_Q', 'e_1_Q', 'e_2_Q', 'iG_1_Q', 'iG_2_Q', 'a_2', 'a_3', 'a_4', 'a_5', 's_1', 's_2', 's_3', 's_4', 'cB_1', 'cB_2', 'g_1', 'g_2', 'g_3', 's_5', 's_6', 's_7', 'cB_3', 'e_1', 'e_2', 'iG_1', 'iG_2', 's_8', 'g_4', 'e_3', 'e_4', 'e_5', 'e_6', 'e_7', 'e_8', 'e_9', 'e_10', 'n_1', 'n_2', 'n_3', 'n_4', 'n_5', 'n_6', 'n_7', 'n_8', 'l_1', 'l_2', 'l_3', 'l_4', 'l_5', 'l_6', 'l_7', 'l_8', 'l_9', 'l_10', 'l_11', 'l_12', 'y_1', 'y_2', 'y_3', 'y_4', 'c_1', 'p_1', 'p_2', 'z_1', 'iG_3', 'iG_4', 'iG_5', 'iG_6', 'iG_7', 'iG_8', 'lC_1', 'lC_2', 'cB_4')) and `root`.collateral_class is not null

... Or executing query directly resulting in a dataframe.

In [0]:
%scala
val assetMapping = legend.query("lcr::lakehouse::assetMapping")
display(assetMapping)

subProduct,reportingEntity,lendableValue,accountingDesignation,treasuryControl,encumbranceType,businessLine,collateralClass,id,marketValue,forwardStartBucket,currency,internalCounterparty,converted,forwardStartAmount,product
Currency and Coin,entity_1,1069776.0,designation_1,true,encumbrance_1,business_1,a_0_Q,0,1444320.0,null,CAD,counterparty_1,true,null,UnencumberedAssets
Level 1,entity_2,3143232.0,designation_2,true,encumbrance_2,business_2,a_1_Q,1,3194640.0,2,USD,counterparty_2,true,2970648.0,Capacity
Level 2a,entity_3,2373336.0,designation_3,false,encumbrance_3,business_3,a_2_Q,2,2460240.0,null,CHF,counterparty_3,true,null,UnrestrictedReserveBalances
Level 2b,entity_4,3684240.0,designation_4,true,encumbrance_4,business_4,a_3_Q,3,3733200.0,null,EUR,counterparty_4,true,null,RestrictedReserveBalances
Non-HQLA,entity_1,3552048.0,designation_1,true,null,business_1,a_4_Q,4,3769920.0,5,CHF,counterparty_1,true,3350088.0,UnsettledAssetPurchases
No Collateral Pledged,entity_2,1341504.0,designation_2,false,encumbrance_2,business_2,a_5_Q,5,1358640.0,6,JPY,counterparty_2,true,946152.0,ForwardAssetPurchases
Rehypothecateable Collateral Unencumbered,entity_3,363528.0,designation_3,true,encumbrance_3,business_3,s_1_Q,6,477360.0,7,USD,counterparty_3,false,140760.0,EncumberedAssets
Unsettled (Regular Way),entity_4,3157920.0,designation_4,true,encumbrance_4,business_4,s_2_Q,7,3206880.0,8,CAD,counterparty_4,true,2892312.0,UnencumberedAssets
Unsettled (Forward),entity_1,3205656.0,designation_1,true,encumbrance_1,business_1,s_3_Q,8,3414960.0,9,USD,counterparty_1,true,2959632.0,Capacity
firm long,entity_2,1177488.0,designation_2,true,encumbrance_2,business_2,s_4_Q,9,1578960.0,10,USD,counterparty_2,true,826200.0,UnrestrictedReserveBalances


Even better, we can go 1 level of abstraction higher and call a legend service where all the necessary transformations were created (and unit tested) on **Legend** in order to transform raw data into **Morphir** ready data assets. By doing so, we comply with the "data contract" implied by the **Morphir** framework with no development overhead. Similarly to logical mapping, we can access its underlying generated SQL...

In [0]:
%scala
legend.generateSql("lcr::services::getInflows")

res6: String = select `root`.product as `product`, `root`.sub_product as `subProduct`, `root`.collateral_class as `collateralClass`, `root`.market_value as `marketValue`, datediff(`assets_maturity_0`.maturity_date, `assets_maturity_0`.purchase_date) as `maturityBucket`, `root`.encumbrance_type as `encumbranceType`, `root`.forward_start_amount as `forwardStartAmount`, `root`.forward_start_bucket as `forwardStartBucket`, `root`.treasury_control as `treasuryControl` from inflows.assets as `root` left outer join inflows.assets_maturity as `assets_maturity_0` on (`root`.id = `assets_maturity_0`.id)

... or directly execute our query resulting in a dataframe

In [0]:
%scala
val inflows = legend.query("lcr::services::getInflows")
display(inflows)

product,subProduct,collateralClass,marketValue,maturityBucket,encumbranceType,forwardStartAmount,forwardStartBucket,treasuryControl
UnencumberedAssets,Currency and Coin,a_0_Q,1444320.0,31,encumbrance_1,null,null,true
Capacity,Level 1,a_1_Q,3194640.0,0,encumbrance_2,2970648.0,2,true
UnrestrictedReserveBalances,Level 2a,a_2_Q,2460240.0,23,encumbrance_3,null,null,false
RestrictedReserveBalances,Level 2b,a_3_Q,3733200.0,28,encumbrance_4,null,null,true
UnsettledAssetPurchases,Non-HQLA,a_4_Q,3769920.0,22,null,3350088.0,5,true
ForwardAssetPurchases,No Collateral Pledged,a_5_Q,1358640.0,19,encumbrance_2,946152.0,6,false
EncumberedAssets,Rehypothecateable Collateral Unencumbered,s_1_Q,477360.0,28,encumbrance_3,140760.0,7,true
UnencumberedAssets,Unsettled (Regular Way),s_2_Q,3206880.0,32,encumbrance_4,2892312.0,8,true
Capacity,Unsettled (Forward),s_3_Q,3414960.0,17,encumbrance_1,2959632.0,9,true
UnrestrictedReserveBalances,firm long,s_4_Q,1578960.0,17,encumbrance_2,826200.0,10,true


## MORPHIR
Finally, we were able to source or data through multiple JOINs operations and necessary transformations without having to write any complex SQL code. The same can be safely passed onto **Morphir** for rule based decisioning and aggregations as set by regulators. Each of those rules have been validated and unit tested through the **Morphir** framework.

In [0]:
%scala
import regulation.us.lcr.inflows.assets.{SparkJobs => Morphir}

import regulation.us.lcr.inflows.assets.{SparkJobs=>Morphir}

Let's execute our first sets of LCR rules for inflows data resulting in a new dataframe

In [0]:
%scala
val lcr = inflows.transform(Morphir.sumToRule)
display(lcr)

Label,value
20(c)(1),7.16869872E9
20(b)(1),3.51023616E9
20(a)(1),1.435490064E10


And safely append or overwrite its results onto a table. That table can be shared across different systems or organizations through delta sharing (see later) or mapped back onto legend to be accessed by end users.

In [0]:
%scala
lcr.write.format("delta").mode("overwrite").saveAsTable("lcr.report")

## TIME TRAVEL
Persisting our reports to Delta format, we benefit from its audit capability allowing users to travel back in time through all its previous versions.

In [0]:
%sql
DESCRIBE HISTORY lcr.report

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
3,2022-11-29T20:00:04.000+0000,3658755248564160,antoine.amend@databricks.com,CREATE OR REPLACE TABLE AS SELECT,"Map(isManaged -> true, description -> null, partitionBy -> [], properties -> {})",null,List(1562452343328116),1129-000847-nuk6je0t,2,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 3, numOutputBytes -> 769)",null,Databricks-Runtime/10.4.x-photon-scala2.12
2,2022-11-29T19:57:29.000+0000,3658755248564160,antoine.amend@databricks.com,CREATE OR REPLACE TABLE AS SELECT,"Map(isManaged -> true, description -> null, partitionBy -> [], properties -> {})",null,List(1562452343328116),1129-000847-nuk6je0t,1,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 3, numOutputBytes -> 769)",null,Databricks-Runtime/10.4.x-photon-scala2.12
1,2022-11-29T19:57:21.000+0000,3658755248564160,antoine.amend@databricks.com,CREATE OR REPLACE TABLE AS SELECT,"Map(isManaged -> true, description -> null, partitionBy -> [], properties -> {})",null,List(1562452343328116),1129-000847-nuk6je0t,0,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 3, numOutputBytes -> 769)",null,Databricks-Runtime/10.4.x-photon-scala2.12
0,2022-11-29T19:56:53.000+0000,3658755248564160,antoine.amend@databricks.com,CREATE OR REPLACE TABLE AS SELECT,"Map(isManaged -> true, description -> null, partitionBy -> [], properties -> {})",null,List(1562452343328116),1129-000847-nuk6je0t,null,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 3, numOutputBytes -> 769)",null,Databricks-Runtime/10.4.x-photon-scala2.12


We access our reports as it was generated at a given point in time (regardless of susequent updates) or at a given version. This ensures audit and compliance requirements whilst guaranteeing strict reproducibility of our output given both our **Legend** and **Morphir** models.

In [0]:
%sql
SELECT * FROM lcr.report
VERSION AS OF 2

Label,value
20(c)(1),1.83790944E9
20(b)(1),8.6119416E8
20(a)(1),3.68918496E9


In [0]:
%sql
SELECT * FROM lcr.report
TIMESTAMP AS OF '2022-11-29T19:57:29.000+0000'

Label,value
20(c)(1),1.83790944E9
20(b)(1),8.6119416E8
20(a)(1),3.68918496E9
